In [ ]:

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import random


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ============================================================
# 1️⃣ CONFIG
# ============================================================
DATASET_PATH = "/content/drive/MyDrive/skeleton_dataset"
MODEL_PATH = "/content/drive/MyDrive/resnet50v2_best.keras"

IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS_HEAD = 5
EPOCHS_FINE = 8

# ============================================================
# 2️⃣ DATA PIPELINE (ImageNet preprocessing is MANDATORY)
# ============================================================
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes
print("Class indices:", train_gen.class_indices)

# ============================================================
# 3️⃣ LOAD PRETRAINED MODEL FROM DRIVE
# ============================================================
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()

# ============================================================
# 4️⃣ STAGE 1 — TRAIN CLASSIFIER HEAD ONLY
# ============================================================
# Freeze EVERYTHING except classifier
for layer in model.layers:
    layer.trainable = False

# Unfreeze classifier layers safely
for layer in model.layers:
    if "dense" in layer.name or "classifier" in layer.name:
        layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        MODEL_PATH,
        monitor="val_loss",
        save_best_only=True,
        verbose=1
    )
]

print("\n🔥 STAGE 1: Training classifier head\n")
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks,
    verbose=1
)

# ============================================================
# 5️⃣ STAGE 2 — FINE-TUNE TOP RESNET BLOCK (conv5)
# ============================================================
# Unfreeze ONLY top ResNet layers (keep BatchNorm frozen)
unfreeze = False
for layer in model.layers:
    if layer.name == "conv5_block1_out":
        unfreeze = True
    if unfreeze and not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n🔥 STAGE 2: Fine-tuning ResNet top block\n")
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_FINE,
    callbacks=callbacks,
    verbose=1
)

# ============================================================
# 6️⃣ FINAL EVALUATION
# ============================================================
from sklearn.metrics import classification_report, confusion_matrix

val_gen.reset()
y_true = val_gen.classes
y_pred = np.argmax(model.predict(val_gen, verbose=1), axis=1)

print("\n📊 CLASSIFICATION REPORT\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_gen.class_indices.keys())
))

print("\n🧮 CONFUSION MATRIX\n")
print(confusion_matrix(y_true, y_pred))

print("\n✅ Fine-tuning completed and model overwritten on Drive.")


In [ ]:
#>>>>>>>>>>>>>>>>>>>QUANTITATIVE ANALYSIS<<<<<<<<<<<<<<<<<<<<<<
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import random

# ===================== PATHS =====================
MODEL_PATH = "/content/drive/MyDrive/resnet50v2_best.keras"
TEST_DIR = "/content/drive/MyDrive/test_dataset"

OUTPUT_IMG = "/content/drive/MyDrive/test_predictions.png"
OUTPUT_CM = "/content/drive/MyDrive/test_confusion_matrix.png"

IMG_SIZE = (224, 224)
THRESHOLD = 0.35   # LOWERED → reduce FN (critical for cheating detection)

# ===================== LOAD MODEL =====================
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded")

# ===================== LOAD TEST IMAGES =====================
image_paths = []
true_labels = []

label_map = {"normal": 0, "suspicious": 1}

for cls in ["normal", "suspicious"]:
    cls_dir = os.path.join(TEST_DIR, cls)
    for fname in os.listdir(cls_dir):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            image_paths.append(os.path.join(cls_dir, fname))
            true_labels.append(label_map[cls])

assert len(image_paths) == 6, "❌ You must have EXACTLY 6 test images"

# ===================== PREPROCESS =====================
images = []
for p in image_paths:
    img = load_img(p, target_size=IMG_SIZE)
    img = img_to_array(img) / 255.0
    images.append(img)

images = np.array(images)
true_labels = np.array(true_labels)

# ===================== PREDICTION =====================
probs = model.predict(images).flatten()   # P(suspicious)
pred_labels = (probs >= THRESHOLD).astype(int)

# ===================== CONFUSION MATRIX =====================
cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(5,4))
sns.heatmap(
    cm, annot=True, fmt="d",
    xticklabels=["normal", "suspicious"],
    yticklabels=["normal", "suspicious"],
    cmap="Blues"
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix (threshold={THRESHOLD})")
plt.savefig(OUTPUT_CM, dpi=150)
plt.show()

print(classification_report(
    true_labels, pred_labels,
    target_names=["normal", "suspicious"]
))

print(f"✅ Confusion matrix saved to {OUTPUT_CM}")

# ===================== VISUALIZE 6 IMAGES =====================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, ax in enumerate(axes.flat):
    img = load_img(image_paths[i], target_size=IMG_SIZE)
    ax.imshow(img)
    ax.axis("off")

    true_cls = "suspicious" if true_labels[i] == 1 else "normal"
    pred_cls = "suspicious" if pred_labels[i] == 1 else "normal"
    prob = probs[i]   # ONLY P(suspicious)

    color = "green" if true_labels[i] == pred_labels[i] else "red"

    ax.set_title(
        f"True: {true_cls}\nPred: {pred_cls}\nProb: {prob:.2f}",
        color=color,
        fontsize=12
    )

plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150)
plt.show()

print(f"✅ Prediction image saved to {OUTPUT_IMG}")
